# Preparation

In [ ]:
! pip install aimsgb

## File selection
Available files:
- POSCAR
- cif file?? 

In [ ]:
from aimsgb import GrainBoundary, Grain
import os

s_input = Grain.from_file("Your file here!!")  ##Direct inputFile

## GrainBoundary(軸, Σ値, 回転方向, s_input(ファイル名))

---

## `GrainBoundary` クラス

構造を入力し、指定した回転軸、シグマ値、粒界面に基づいて粒界モデルを構築します。

### **使用法**

```python
GrainBoundary(axis, sigma, face, s_input, uc_a=1, uc_b=1)
```
### `GrainBoundary` 引数リスト

- **`axis`**: **回転軸**
  - 粒界の基準となる回転軸を指定します。
- **`sigma`**: **シグマ値 ($\Sigma$)**
  - CSL理論に基づいたシグマ値を指定します。
- **`face`**: **粒界面**
  - 粒界を形成する結晶面（ミラー指数など）を指定します。
- **`s_input`**: **結晶構造**
  - ベースとなる結晶構造データを入力します。
- **`uc_a`**: **左側バルク倍率**（初期値: `1`）
  - 粒界の左側の領域を何倍に拡張するかを指定します（対称性維持のため）。
- **`uc_b`**: **右側バルク倍率**（初期値: `1`）
  - 粒界の右側の領域を何倍に拡張するかを指定します。

粒界エネルギーの安定性のために、uc_aとuc_bを同じ値に設定することが推奨されます。

In [ ]:
## 以下はaimsgbのチュートリアルです。
axis = [1,1,0]
sigma = 3
face= [1,1,0]

gb = GrainBoundary(axis, sigma, face, s_input,uc_a=2, uc_b=1)

structure = Grain.stack_grains(gb.grain_a, gb.grain_b, direction=gb.direction)
folder_path = "masn001\sigma"+str(sigma)+"_"+str(face)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
else:
    folder_path=folder_path+"_2"
    os.makedirs(folder_path)

        # Save the structure variable as a POSCAR file
structure.to(filename=f"{folder_path}\POSCAR");

## Miscellious

### GBInformation
Grain Boundaryの情報を可視化するのにとても便利。
CSL(Coincidence Site Lattice)が各Σ値に対してどのようなGB平面、CSL行列を示すかを出力してくれる。

#### 基本的な使い方
`GBInformation(axis, max_sigma).__str__()`

- **`axis`**: 回転軸を指定します。
- **`max_sigma`**: シグマ値の最大値を指定します。
    - この値までCSL（Coincidence Site Lattice）情報を計算し、結果を返します。

In [2]:
from aimsgb import GBInformation

print(GBInformation([0,0,1], 20).__str__())

Grain boundary information for rotation axis: 001
Show the sigma values up to 20 (Note: * means twist GB, Theta is the rotation angle)
|  Sigma  |  Theta  | GB plane   | CSL matrix   |
|---------+---------+------------+--------------|
|    5    |  53.13  | (2 -1 0)   | 2  1  0      |
|         |         | (1 2 0)    | -1  2  0     |
|         |         | (0 0 1)*   | 0  0  1      |
|    5    |  36.87  | (3 -1 0)   | 3  1  0      |
|         |         | (1 3 0)    | -1  3  0     |
|         |         | (0 0 1)*   | 0  0  1      |
|   13    |  67.38  | (3 -2 0)   | 3  2  0      |
|         |         | (2 3 0)    | -2  3  0     |
|         |         | (0 0 1)*   | 0  0  1      |
|   13    |  22.62  | (5 -1 0)   | 5  1  0      |
|         |         | (1 5 0)    | -1  5  0     |
|         |         | (0 0 1)*   | 0  0  1      |
|   17    |  28.07  | (4 1 0)    | 4 -1  0      |
|         |         | (-1 4 0)   | 1  4  0      |
|         |         | (0 0 1)*   | 0  0  1      |
|   17    |  61

#### MASnI3の使用例

In [ ]:
from aimsgb import GrainBoundary, Grain
import os

s_input = Grain.from_file("POSCAR_MASnI3")  ##Direct inputFile
axis = [1,1,0]
sigma = 3
face= [1,1,0]

gb = GrainBoundary(axis, sigma, face, s_input,uc_a=2, uc_b=1)

structure = Grain.stack_grains(gb.grain_a, gb.grain_b, direction=gb.direction)
folder_path = "masn001\sigma"+str(sigma)+"_"+str(face)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
else:
    folder_path=folder_path+"_2"
    os.makedirs(folder_path)

        # Save the structure variable as a POSCAR file
structure.to(filename=f"{folder_path}\POSCAR");

### `get_allgb` 関数の概要と動作説明

この関数は、`aimsgb` ライブラリを使用して、特定の回転軸（`axis`）に対して定義される複数の結晶粒界（Grain Boundary, GB）モデルを自動生成し、それぞれの構造ファイル（POSCAR）と設定条件（Condition.txt）をディレクトリごとに保存します。

#### 1. 関数の引数
- **`axis`**: 回転軸を指定するリストまたは配列（例: `[0, 0, 1]`）。
- **`s_input`**: 元となる単結晶構造ファイルのパス（例: `'POSCAR_unit'`）。

#### 2. 主な動作フロー
1.  **GB情報の取得**: `GBInformation(axis, 13)` を呼び出し、指定された回転軸で $\Sigma \le 13$ までの粒界情報を取得します。
2.  **テキスト解析**: 取得したテーブル形式の文字列をパースし、$\Sigma$ 値、回転角（rotate）、および面指数（buffer/plane）を抽出します。
3.  **構造の生成**:
    - `Grain.from_file(s_input)` で入力構造を読み込みます。
    - `GrainBoundary` クラスを用いて、$\Sigma$ 値や面指数に基づいた粒界オブジェクトを作成します。
    - `Grain.stack_grains` により、2つの結晶粒（grain_a, grain_b）をスタックして粒界構造を構築します。
4.  **ファイルの保存**:
    - `masn001\sigma{Sigma値}_{面指数}` という名称のフォルダを自動作成します（重複する場合は `_2` を付与）。
    - フォルダ内に粒界構造を **`POSCAR`** として保存します。
    - フォルダ内に計算条件（回転角など）を **`Condition.txt`** として保存します。

#### 3. 出力結果
実行すると、カレントディレクトリに以下のようなディレクトリ構造が生成されます。

```text
masn001/
  ├── sigma5_[1, 2, 0]/
  │     ├── POSCAR          (粒界構造ファイル)
  │     └── Condition.txt   (回転角などの情報)
  ├── sigma5_[1, 3, 0]/
  │     ├── POSCAR
  │     └── Condition.txt
  └── ...
```

#### 4. 注意点
- **ディレクトリセパレータ**: コード内で `\` が使用されているため、Windows環境を想定しています。
- **依存ライブラリ**: `aimsgb` および `pymatgen` がインストールされている必要があります。
- **解析ロジック**: `GBInformation` の出力形式に依存してパースを行っているため、ライブラリのバージョンアップ等で出力形式が変わると修正が必要になる可能性があります。



In [2]:
def get_allgb(axis,s_input):
    
    from aimsgb import GrainBoundary, Grain
    import os
    from aimsgb import GBInformation

    ## Collect the GB information

    strings=GBInformation(axis, 13).__str__()

    llist=strings.split('\n')
    
    n=3
    sigma =0
    
    for i in range(len(llist)-4):
        s=str(llist[i+4]).strip()
        cut=s.split("|")

        temp=["a","a","a","a"]

        for i in range(len(cut)-2):
            string=cut[i+1].lstrip()
            string=string.rstrip()
            string=string.replace("(", "")
            string=string.replace(")", "")
            string=string.rstrip("*")
    
        ##ifで∑をのこす
            #print(string)

            if i==0 and n!=3:
                #print(n)
                pass
            
            elif n==3 and i==0:
                sigma=int(string)
                n=0
                
            elif n==0 and i==1:
                rotate=float(string)
            
            elif i==2:
                cut2=string.split(" ")
                buffer=[1,1,1]
                for i in range(3):
                    buffer[i]=int(cut2[i]) 
            else:
                temp[i]=string
    
            # Make the GB
        s_input2 = Grain.from_file(s_input)
        n+=1
        #print(temp[0], "   buffer:", buffer)

        gb = GrainBoundary(axis, sigma, buffer, s_input2,uc_a=1, uc_b=1)
        structure = Grain.stack_grains(gb.grain_a, gb.grain_b, direction=gb.direction)

            # Create the folder if it doesn't exist
        folder_path = "masn001\sigma"+str(sigma)+"_"+str(buffer)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        else:
            folder_path=folder_path+"_2"
            os.makedirs(folder_path)

        # Save the structure variable as a POSCAR file
        structure.to(filename=f"{folder_path}\POSCAR");
        # Save the temp[i] variable content to Condition.txt file
        condition_path = os.path.join(folder_path, 'Condition.txt')
        temp[1]=str(rotate)
        with open(condition_path, 'w') as f:
            for item in temp:
                f.write("%s\n" % item)
        
        ##カウントして、3回に一度だけ∑を更新する


#### 使用例

In [3]:
get_allgb([0,0,1],"POSCAR_MASnI3")

In [63]:
def getmp_allgb(axis,s_input):
    
    from aimsgb import GrainBoundary, Grain
    import os
    from aimsgb import GBInformation

    ## Collect the GB information

    strings=GBInformation(axis, 13).__str__()

    llist=strings.split('\n')

    s=str(llist[7]).strip()
    cut=s.split("|")

    temp=["a","a","a","a"]

    for i in range(len(cut)-2):
        string=cut[i+1].lstrip()
        string=string.rstrip()
        string=string.replace("(", "")
        string=string.replace(")", "")
    
        ##ifで∑をのこす

        if i==1 and string=="":
            break

        elif i==2:
            cut2=string.split(" ")
            buffer=[1,1,1]
            for i in range(3):
                buffer[i]=int(cut2[i]) 
        else:
            temp[i]=string
    
         # Make the GB
    s_input = Grain.from_mp_id("mp-2815") 
    #gb = GrainBoundary(axis, int(temp[0]), temp[2], s_input)
    temp[2]=temp[2].replace(" ",", ")
    print(temp[2])
    gb = GrainBoundary(axis, int(temp[0]), buffer, s_input)
    structure = Grain.stack_grains(gb.grain_a, gb.grain_b, direction=gb.direction)

        # Create the folder if it doesn't exist
    folder_path = "sigma"+str(temp[0])+"_"+str(face)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

        # Save the structure variable as a POSCAR file
    structure.to(filename=f"{folder_path}\POSCAR");

In [ ]:
s_input = Grain.from_mp_id("mp-1094059")    #### Retrived from Materials project.
getmp_allgb([1,1,0],"mp-2815")

Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<?, ?it/s]
c:\Users\ajsf\Documents\ito\.venv\Lib\site-packages\aimsgb\grain.py:263: UserWarning: The lattice system of the grain is not orthogonal. The lattice will be forced to be orthogonal. **Note that this breaks inherent symmetries of the grain.**
  warnings.warn("The lattice system of the grain is not orthogonal. The lattice "


a


## CsPbI3 TO MAPbI3

In [ ]:
def cstoma(file):
    f=open(file,"r")
    poscar_line=f.readlines()
    atom=poscar_line[5].split( )
    cood=poscar_line[6].split( )
    x=poscar_line[2].split( )[0]
    y=poscar_line[3].split( )[1]
    z=poscar_line[4].split( )[2]
    csnum=0 ##csは何番目？
    for i in range(len(atom)):
        if "Cs"==atom[i]:
            csnum=i
    
    atomnum=0 ##Csに至るまでの原子の数
    if csnum!=0:
        for i in range(csnum):atomnum+=int(cood[i])
    
    ##8 line kara cood
    for i in range(cood[csnum]):
        c_cood=[]
        h_cood=[]
        n_cood=[]
        atcood=poscar_line[i+8+csnum].split( ) ##each atom cood
        
        c_cood.append()

In [4]:
import numpy as np
import gb_code.csl_generator as csl

# for example: [1, 0, 0], [1, 1, 0] or [1, 1, 1]
axis = np.array([-1, 1, 0])

# list Sigma boundaries < 50
csl.print_list(axis, 50)

Sigma:     1  Theta:   0.00 
Sigma:     3  Theta:  70.53 
Sigma:     9  Theta:  38.94 
Sigma:    11  Theta:  50.48 
Sigma:    17  Theta:  86.63 
Sigma:    19  Theta:  26.53 
Sigma:    27  Theta:  31.59 
Sigma:    33  Theta:  20.05 
Sigma:    41  Theta:  55.88 
Sigma:    43  Theta:  80.63 


In [6]:
from numpy import array, dot, degrees, cross
# pick a sigma for this axis, ex: 19.
sigma = 3

theta, m, n = csl.get_theta_m_n_list(axis, sigma)[0]

R = csl.rot(axis, theta)

# Minimal CSL cells. The plane orientations and the orthogonal cells
# will be produced from these original cells.
M1, M2 = csl.Create_minimal_cell_Method_1(sigma, axis, R)

print('Angle:', degrees(theta), '\n', 'Sigma:', sigma, '\n', 
      'Minimal cells:', '\n', M1, '\n', M2, '\n')  

Angle: 70.52877936550931 
 Sigma: 3 
 Minimal cells: 
 [[-1 -1 -1]
 [-1  0  1]
 [-1  1  0]] 
 [[-1  0 -1]
 [-1  1  1]
 [ 1  1  0]] 

